In [1]:
import numpy as np
import numpy as npX
import pandas as pd
import seaborn as sns
import json

In [2]:
vec = pd.read_csv("feature_vec.csv")
#df1 = df.set_index('timestamp')

res = pd.read_csv("results_merged.csv")

res.head()

,tweet_id,priority,postCategories_x,positive_sentiment,negative_sentiment,numb_of_questions,tweet_length,userFollowersCount,userFriendsCount,user_list_count,dict_precision,dict_recall,dict_f_measure,result
0,2.115660e+17,0.75,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,2.116544e+17,0.50,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
2,2.116813e+17,0.25,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,2.116856e+17,0.25,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
4,2.118770e+17,0.25,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5


In [3]:
df = res

In [9]:
with open('../../../data/raw/data/2020/2020-A/types/TRECIS-2020-ITypes-Task1.json', encoding='utf-8') as ontologyJSONFile:    
    ontologyJSON = json.load(ontologyJSONFile)

informationTypes2Index = {} # category -> numerical index
informationTypesShort2Index = {} # category short form (e.g. Report-EmergingThreats vs. EmergingThreats) -> numerical index
informationTypeIndex = 0
for informationTypeJSON in ontologyJSON["informationTypes"]:
    informationTypeId = informationTypeJSON["id"]
    informationTypes2Index[informationTypeId] = informationTypeIndex
    informationTypesShort2Index[informationTypeId.split("-")[1]] = informationTypeIndex
    informationTypeIndex = informationTypeIndex + 1
informationTypes2Index

{'Request-GoodsServices': 0,
 'Request-SearchAndRescue': 1,
 'Request-InformationWanted': 2,
 'CallToAction-Volunteer': 3,
 'CallToAction-Donations': 4,
 'CallToAction-MovePeople': 5,
 'Report-FirstPartyObservation': 6,
 'Report-ThirdPartyObservation': 7,
 'Report-Weather': 8,
 'Report-Location': 9,
 'Report-EmergingThreats': 10,
 'Report-NewSubEvent': 11,
 'Report-MultimediaShare': 12,
 'Report-ServiceAvailable': 13,
 'Report-Factoid': 14,
 'Report-Official': 15,
 'Report-News': 16,
 'Report-CleanUp': 17,
 'Report-Hashtags': 18,
 'Report-OriginalEvent': 19,
 'Other-ContextualInformation': 20,
 'Other-Advice': 21,
 'Other-Sentiment': 22,
 'Other-Discussion': 23,
 'Other-Irrelevant': 24}

In [10]:
eventidTopicidMap =	{
  "athensEarthquake2020": "35",
  "baltimoreFlashFlood2020": "36",
  "brooklynBlockPartyShooting2020": "37",
  "daytonOhioShooting2020": "38",
  "elPasoWalmartShooting2020": "39",
  "gilroygarlicShooting2020": "40",
  "hurricaneBarry2020": "41",
  "indonesiaEarthquake2020": "42",
  "keralaFloods2020": "43",
  "myanmarFloods2020": "44",
  "papuaNewguineaEarthquake2020": "45",
  "siberianWildfires2020": "46",
  "typhoonKrosa2020": "47",
  "typhoonLekima2020": "48",
  "whaleyBridgeCollapse2020": "49"
}
eventidTopicidMap

{'athensEarthquake2020': 'TRECIS-CTIT-H-Test-035',
 'baltimoreFlashFlood2020': 'TRECIS-CTIT-H-Test-036',
 'brooklynBlockPartyShooting2020': 'TRECIS-CTIT-H-Test-037',
 'daytonOhioShooting2020': 'TRECIS-CTIT-H-Test-038',
 'elPasoWalmartShooting2020': 'TRECIS-CTIT-H-Test-039',
 'gilroygarlicShooting2020': 'TRECIS-CTIT-H-Test-040',
 'hurricaneBarry2020': 'TRECIS-CTIT-H-Test-041',
 'indonesiaEarthquake2020': 'TRECIS-CTIT-H-Test-042',
 'keralaFloods2020': 'TRECIS-CTIT-H-Test-043',
 'myanmarFloods2020': 'TRECIS-CTIT-H-Test-044',
 'papuaNewguineaEarthquake2020': 'TRECIS-CTIT-H-Test-045',
 'siberianWildfires2020': 'TRECIS-CTIT-H-Test-046',
 'typhoonKrosa2020': 'TRECIS-CTIT-H-Test-047',
 'typhoonLekima2020': 'TRECIS-CTIT-H-Test-048',
 'whaleyBridgeCollapse2020': 'TRECIS-CTIT-H-Test-049'}

In [7]:
priority_scorer = {
    'Critical' : 0.75,
    'High' : 0.75,
    'Medium' : 0.50,
    'Low' : 0.25,
    'Unknown' : 0.0,
}

categoriser = {
    'Critical' : 0.75,
    'High' : 0.75,
    'Medium' : 0.50,
    'Low' : 0.25,
    'Unknown' : 0.0,
}

# write to .run file
with open("marks.run" , "w") as out_file:
    for row in df.drop_duplicates(subset="tweet_id").itertuples():
        print("row:", row)
        content = [
            "TRECIS-CTIT-H-Test-046",
            #row.inc_id,     # category
            "Q0",           # static `Q0`
            row.tweet_id,   # 
            getattr(row, 'Index'),
            row.priority, 
            #row.category,
            informationTypes2Index[row.postCategories_x], #str(row.postCategories_x).replace("'", '"'),
            "marksrun"
        ]
        out_file.write("\t".join([str(x) for x in content]) + "\n")

row: Pandas(Index=0, tweet_id=2.115659744224256e+17, priority=0.75, postCategories_x=19.0, positive_sentiment=0.0, negative_sentiment=0.0, numb_of_questions=0.0, tweet_length=0.0, userFollowersCount=0.0, userFriendsCount=0.0, user_list_count=0.0, dict_precision=0.0, dict_recall=0.0, dict_f_measure=0.0, result=5)


KeyError: 19.0